# 2. SVM

In [1]:
import numpy as np
import pandas as pd

from sklearn import svm, metrics
from sklearn.model_selection import StratifiedKFold

## 2.1 Open the dataset

In [2]:
BASE_DIR = '..'
DATA_DIR = f'{BASE_DIR}/data'

In [3]:
sample_df = pd.read_csv(f'{DATA_DIR}/sample.csv')

In [4]:
sample_df.head()

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_775,pixel_776,pixel_777,pixel_778,pixel_779,pixel_780,pixel_781,pixel_782,pixel_783,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0


In [5]:
y = sample_df.pop('target').astype(int).values
X = sample_df.values

## Fit a simple SVM Classifier

In [6]:
def load_model(C=1):
    model = svm.SVC(kernel='linear', C=C)
    
    return model

In [7]:
def train_evaluate(model, X_train, y_train, X_test, y_test):
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)

    return metrics.accuracy_score(y_test, y_pred)

In [8]:
n_folds = 10

In [9]:
kFold = StratifiedKFold(n_splits=n_folds)

In [10]:
results = {}

In [11]:
for C in [0.01, 0.1, 1, 100, 1000, 10000]:
    
    for i, (train, test) in enumerate(kFold.split(X, y)):
        scores = []
        print(f'Running fold {i}... on model with param C = {C}')
        model = load_model(C)
        acc = train_evaluate(model, X[train], y[train], X[test], y[test])
        scores.append(acc)

        scores = np.array(scores)

    print(f"Accuracy: {scores.mean():.4f} (+/- {scores.std():.4f})")
    results[C] = scores.mean()

Running fold 0... on model with param C = 0.01
Running fold 1... on model with param C = 0.01
Running fold 2... on model with param C = 0.01
Running fold 3... on model with param C = 0.01
Running fold 4... on model with param C = 0.01
Running fold 5... on model with param C = 0.01
Running fold 6... on model with param C = 0.01
Running fold 7... on model with param C = 0.01
Running fold 8... on model with param C = 0.01
Running fold 9... on model with param C = 0.01
Accuracy: 0.8000 (+/- 0.0000)
Running fold 0... on model with param C = 0.1
Running fold 1... on model with param C = 0.1
Running fold 2... on model with param C = 0.1
Running fold 3... on model with param C = 0.1
Running fold 4... on model with param C = 0.1
Running fold 5... on model with param C = 0.1
Running fold 6... on model with param C = 0.1
Running fold 7... on model with param C = 0.1
Running fold 8... on model with param C = 0.1
Running fold 9... on model with param C = 0.1
Accuracy: 0.8222 (+/- 0.0000)
Running fo

In [12]:
results

{0.01: 0.8,
 0.1: 0.8222222222222222,
 1: 0.8222222222222222,
 100: 0.8222222222222222,
 1000: 0.8222222222222222,
 10000: 0.8222222222222222}

## Data Augmentation

In [13]:
def augment_fold(X, y, aug_factor=5):
    from keras.preprocessing.image import ImageDataGenerator
    
    image_generator = ImageDataGenerator(width_shift_range=0.1)
    
    image_generator.fit(X.reshape(-1, 28, 28, 1), augment=True)
    
    np_iter = image_generator.flow(X.reshape(-1, 28, 28, 1), y, batch_size=500)
    
    X_aug_lst = [X]
    y_aug_lst = [y]
    
    for i in range(aug_factor):
        X_temp, y_temp = np_iter.next()
        X_temp = X_temp.reshape(len(X_temp), -1)
        X_aug_lst.append(X_temp)
        y_aug_lst.append(y_temp)
        
    X_aug = np.vstack(X_aug_lst)
    y_aug = np.concatenate(y_aug_lst)
    
    assert X_aug.shape[0] == y_aug.shape[0], f"X_aug and y_aug shapes mismatch {X_aug.shape[0]} - {y_aug.shape[0]}"
    
    X_aug = np.vstack(X_aug_lst)
    y_aug = np.concatenate(y_aug_lst)
    
    return X_aug, y_aug

In [14]:
n_folds = 10

In [15]:
kFold = StratifiedKFold(n_splits=n_folds)

In [16]:
results = {}

In [17]:
X_a, y_a = augment_fold(X, y, aug_factor=4)

Using TensorFlow backend.
/home/miguel/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [18]:
for C in [0.01, 0.1, 1, 100, 1000, 10000]:
    
    for i, (train, test) in enumerate(kFold.split(X_a, y_a)):
        scores = []
        print(f'Running fold {i}... on model with param C = {C}')
        model = load_model(C)
#         X_aug, y_aug = augment_fold(X[train], y[train], aug_factor=4)
        acc = train_evaluate(model, X_a[train], y_a[train], X_a[test], y_a[test])
        scores.append(acc)

        scores = np.array(scores)

    print(f"Accuracy: {scores.mean():.4f} (+/- {scores.std():.4f})")
    results[C] = scores.mean()

Running fold 0... on model with param C = 0.01
Running fold 1... on model with param C = 0.01
Running fold 2... on model with param C = 0.01
Running fold 3... on model with param C = 0.01
Running fold 4... on model with param C = 0.01
Running fold 5... on model with param C = 0.01
Running fold 6... on model with param C = 0.01
Running fold 7... on model with param C = 0.01
Running fold 8... on model with param C = 0.01
Running fold 9... on model with param C = 0.01
Accuracy: 0.8866 (+/- 0.0000)
Running fold 0... on model with param C = 0.1
Running fold 1... on model with param C = 0.1
Running fold 2... on model with param C = 0.1
Running fold 3... on model with param C = 0.1
Running fold 4... on model with param C = 0.1
Running fold 5... on model with param C = 0.1
Running fold 6... on model with param C = 0.1
Running fold 7... on model with param C = 0.1
Running fold 8... on model with param C = 0.1
Running fold 9... on model with param C = 0.1
Accuracy: 0.9717 (+/- 0.0000)
Running fo

In [19]:
results

{0.01: 0.8866396761133604,
 0.1: 0.97165991902834,
 1: 0.9757085020242915,
 100: 0.9757085020242915,
 1000: 0.9757085020242915,
 10000: 0.9757085020242915}

### Train

In [20]:
from sklearn.externals import joblib

In [21]:
model = load_model()

In [22]:
model.fit(X_a, y_a)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [23]:
joblib.dump(model, 'svm_v1.pkl') 

['svm_v1.pkl']

In [24]:
X.shape

(500, 784)

In [25]:
model.predict([X[0]])

array([8])